In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = '/kaggle/input/m5-forecasting-accuracy'
train_sales = pd.read_csv(f'{path}/sales_train_validation.csv')
calendar = pd.read_csv(f'{path}/calendar.csv')
submission = pd.read_csv(f'{path}/sample_submission.csv')
sell_prices = pd.read_csv(f'{path}/sell_prices.csv')
train_sales.shape,calendar.shape,submission.shape,sell_prices.shape

### train_sale：销售训练数据
- HOBBIES_1_001：item_id列中:类别--出售部门--编号
- dept_id: 商品出售部门
- cat_id: 商品的类别
- store_id：哪个商店出售
- state_id：商品所在州
- d_1-d_1913：第一天至1913天，每天的商品销售情况

In [ ]:
train_sales.head()

In [ ]:
train_sales['item_id'].value_counts()

### calendar：日历
- date：2011.1.29--2016.6.19
- wm_yr_wk:wm代号沃尔玛，标识1无意义，2-3代表年份，最后两位代表今年第几周
- d：对于日期的编号
- event_name_1:在那一天发生的第一场大事件
- event_type_1:在那一天发生的第一场大事件类型
- event_name_2:在那一天发生的第二场大事件
- event_type_2:在那一天发生的第二场大事件的类型
- snap_CA:加州的商店是否可以在那一天可以使用福利券
- snap_TX:得克萨斯州的商店是否可以在那一天使用福利券
- snap_WI:威斯康星州的商店是否可以在那一天使用福利券

In [ ]:
calendar.head()

### submission：提交格式
- 提交训练集中往后1-28天的预测值
- 根据训练集上的时间长度，公共榜单上的应该是d_1914天至d_1941天的商品销量预测值
- 公共榜单关闭后，d_1914天至d_1941的真实值会公布，预测的时间往后延续

In [ ]:
submission.head()

### sale_price：销售价格
- store_id：商店的ID
- item_id：商品的编号
- wm_yr_wk：训练集中相同
- sell_price: 销售价格

In [ ]:
sell_prices.head()

## 一、数据EDA
### 1.1 基本信息

In [ ]:
print(train_sales['item_id'].value_counts())
train_sales.loc[train_sales['item_id']=='HOBBIES_1_001',:]

In [ ]:
train_sales['cat_id'].value_counts()

#### 结论
- 数据来源分布: 同一个item_id（商品类别—出售门店-出售编号），从不同地方选择10条记录
- 商品总共3049种,总共三大类产品

### 1.2 价格信息

In [ ]:
# 价格信息：每个item_id对应的价格是波动的
tmp = sell_prices.loc[sell_prices['item_id']=='HOBBIES_1_001',:]
print(tmp['sell_price'].value_counts())
tmp

In [ ]:
fig = plt.figure(figsize=(16,6))
fig.set(alpha=0.2) 

plt.subplot2grid((1,2),(0,0)) 
# 查看每个商店的价格波动

item_prices = sell_prices.loc[sell_prices['item_id'].isin(['HOBBIES_1_001','FOODS_1_016'])]    
for i,item_id in enumerate(['HOBBIES_1_001','FOODS_1_016']):
    plt.subplot2grid((1,2),(0,i))
    tmp = item_prices[item_prices['item_id']==item_id]
    for i,store_id in enumerate(tmp['store_id'].unique()):
        store_sell_price = tmp.loc[tmp['store_id'] == store_id]
        plt.plot(store_sell_price['wm_yr_wk'].values,store_sell_price['sell_price'].values,label=store_id)
        
    plt.title(f'{item_id} sell_price trend')
    plt.legend()

In [ ]:
# 查看该商品下所有类目价格
ca_1_prices = sell_prices.loc[sell_prices['store_id'] == 'CA_1']
ca_1_prices['dept_id'] = ca_1_prices['item_id'].apply(lambda x: x[:-4])

plt.figure(figsize=(12, 6))
for dept_id in ca_1_prices['dept_id'].unique():
    small_df = ca_1_prices.loc[ca_1_prices['dept_id'] == dept_id]
    grouped = small_df.groupby(['wm_yr_wk'])['sell_price'].mean()
    plt.plot(grouped.index, grouped.values, label=dept_id)
plt.legend(loc=(1.0, 0.5))
plt.title('CA_1 mean sell prices by dept');

In [ ]:
ca_prices = sell_prices.loc[sell_prices['store_id'].str.contains('CA')]
plt.figure(figsize=(12, 6))
for d in ca_prices['store_id'].unique():
    small_df = ca_prices.loc[ca_prices['store_id'] == d]
    grouped = small_df.groupby(['wm_yr_wk'])['sell_price'].mean()
    plt.plot(grouped.index, grouped.values, label=d)
plt.legend(loc=(1.0, 0.5))
plt.title('Mean sell prices by store in CA');

#### 结论
- 在不同的时间周期，价格波动的，部分类目存在打折情况
- 不同的商店之间定价也存在微小的差异，差别不大
- 价格的高低与该部门的销量存在一定的相关，但不明显
- 不同的店铺价格有差异，并且价格的波动是随时间变化的

### 3、销量信息

In [ ]:
# 销量信息：不同商品信息90天日均存在很大的波动
fig = plt.figure(figsize=(16,16))
fig.set(alpha=0.2) 
# 设置子图位置

plt.subplot2grid((3,1),(0,0)) 
for i in range(10):
    plt.plot(train_sales.loc[train_sales['item_id']=='HOBBIES_1_001'].iloc[i,6:].rolling(90).mean().values,\
            label=train_sales.loc[train_sales['item_id']=='HOBBIES_1_001'].iloc[i,5])
    plt.title('HOBBIES_1_001 sales, rolling mean 90 days')
    plt.legend();

plt.subplot2grid((3,1),(1,0))
for j in range(10):
    plt.plot(train_sales.loc[train_sales['item_id']=='FOODS_1_016'].iloc[i,6:].rolling(90).mean().values,\
            label=train_sales.loc[train_sales['item_id']=='FOODS_1_016'].iloc[i,5])
    plt.title('FOODS_1_016 sales, rolling mean 90 days')
    plt.legend();

plt.subplot2grid((3,1),(2,0))
tmp= train_sales[train_sales['item_id'].isin(['FOODS_1_016','HOBBIES_1_001'])].groupby('item_id',as_index=False).mean()
for i in ['FOODS_1_016','HOBBIES_1_001']:
    plt.plot(tmp.loc[tmp['item_id']==i].iloc[0,1:].rolling(90).mean().values,label=i)
plt.title('item_id mean trend')
plt.legend();

#### 结论
- 商店的日均销量随时间存在较大的波动，容易受季节性或活动影响，不同的商品在不同店铺的销量同样存在较大差别
- 图三针对单一商品进行日均销量统计，发现不同商品所属的类目销量之前同样差异很大

### 4、店铺类目-dept信息

In [ ]:
# 具体到某个店
cat_1_sales = train_sales.loc[train_sales['store_id'] == 'CA_1']
pd.crosstab(cat_1_sales['cat_id'],cat_1_sales['dept_id'])

In [ ]:
dept_sum = cat_1_sales.groupby('dept_id').sum()
plt.figure(figsize=(16,12))
for i,row in dept_sum.iterrows():
    plt.plot(row.values,label=i)
plt.title(f'CA1 sales by dept')
plt.legend()
plt.show()

#### 结论
- 不同的类目下的产品日销量差别较大，FOODS_3销量最高，存在一定的小周期性
- 统一类目下不同部门日销量同样较大，有些呈现一定周期性，有些存在线下上升趋势

### 5、时间信息

In [ ]:
fig, ax = plt.subplots(4,3,figsize=(16,22))
plt.subplots_adjust(wspace=0.2,hspace=0.8)
for i, var in enumerate(["year", "weekday", "month", "event_name_1", "event_name_2", 
                         "event_type_1", "event_type_2", "snap_CA", "snap_TX", "snap_WI"]):
    i += 1
    plt.subplot(4,3,i)
    g = sns.countplot(calendar[var])
    g.set_xticklabels(g.get_xticklabels(), rotation=45)
    g.set_title(var)

### 后续
- 希望对大家有帮助
- 持续更新中